In [4]:
!pip install arxiv # For research paper platform

  Using cached arxiv-2.1.3-py3-none-any.whl (11 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed.
    Running setup.py install for sgmllib3k: started
    Running setup.py install for sgmllib3k: finished with status 'done'


You should consider upgrading via the 'D:\Projects Folder\LangChain_Series\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install wikipedia

You should consider upgrading via the 'D:\Projects Folder\LangChain_Series\venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Using cached wikipedia-1.4.0.tar.gz (27 kB)
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
    Running setup.py install for wikipedia: started
    Running setup.py install for wikipedia: finished with status 'done'


In [20]:
!pip install langchainhub

  Using cached types_requests-2.32.0.20240712-py3-none-any.whl (15 kB)


You should consider upgrading via the 'D:\Projects Folder\LangChain_Series\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Creating Wikipedia Tool

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [9]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [10]:
wiki_tool.name

'wikipedia'

In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027AD07A6560>)

In [12]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [13]:
retriever_tool.name

'langsmith_search'

In [15]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [16]:
tools=[wiki_tool,arxiv_tool,retriever_tool]

In [17]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Projects Folder\\LangChain_Series\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000027A894ED480>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.

In [18]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3", temperature=0)

In [40]:
from langchain import hub
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

In [41]:
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt1 = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful assistant"),
#         MessagesPlaceholder("chat_history", optional=True),
#         ("human", "{input}"),
#         MessagesPlaceholder("agent_scratchpad"),
#     ]
# )
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

d:\Projects Folder\LangChain_Series\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [42]:
### Agents
from langchain.agents import create_openai_tools_agent, create_react_agent
# agent=create_openai_tools_agent(llm,tools,prompt)
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

In [43]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'chat_history': '', 'instructions': 'You are an assistant.', 'tools': "wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS

In [37]:
# # Ensure that the prompt returns a ChatPromptTemplate object
# from langchain_core.prompts import ChatPromptTemplate
# if not isinstance(prompt, ChatPromptTemplate):
#     raise ValueError("Prompt should be a ChatPromptTemplate instance.")

In [44]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: langsmith_search
Action Input: query='Langsmith', retriever='BaseRetriever', document_prompt='BasePromptTemplate', document_separator='\n\n', callbacks=None= traceable(async (user_input) => {    const result = await client.chat.completions.create({        messages: [{ role: "user", content: user_input }],        model: "gpt-3.5-turbo",    });    return result.choices[0].message.content;});await pipeline("Hello, world!")// Out: Hello there! How can I assist you today?View a sample output trace.Learn more about tracing in the how-to guides.5. Run your first evaluation​Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.PythonTypeScriptfrom langsmith import Clientfrom langsmith.evaluation import evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset 

{'input': 'Tell me about Langsmith', 'output': 'Welcome to Langsmith!'}

In [45]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: arxiv
Action Input: "1605.08386

d:\Projects Folder\LangChain_Series\venv\lib\site-packages\langchain_community\utilities\arxiv.py:152: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThought: Do I need to use a tool? Yes
Action: arxiv
Action Input: "1605.08386

d:\Projects Folder\LangChain_Series\venv\lib\site-packages\langchain_community\utilities\arxiv.py:152: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloI'll continue the conversation!

Thought: Do I need to use a tool? No
Final Answer: The paper "1605.08386" is about Heat-bath random walks with Markov bases, published in 2016 by Caprice Stanley and Tobias Windisch. It studies graphs on lattice points whose edges come from a finite set of allowed moves.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper "1605.08386" is about Heat-bath random walks with Markov bases, published in 2016 by Caprice Stanley and Tobias Windisch. It studies graphs on lattice points whose edges come from a finite set of allowed moves.'}

https://medium.com/@manojmukherjee777/advanced-agent-functionality-with-ollama-and-llama-3-in-langchain-95a80ded5489